#### Get online data from Robert Shiller here: http://www.econ.yale.edu/~shiller/data.htm
#### Innlegget mitt: https://www.trifektum.no/data/investering-i-aksjer-nar-inflasjon-er-hoy/

### Trenger data for:
- aksjeavkastning i real termer (S&P)
- Inflasjon (USA)
- Shiller PE i real termer (PE10)

##### og videre beregne:
- 1 års endring CPI
- annualisert volatilitet aksjer
- annualisert volatilitet inflasjon

In [1]:
import pandas as pd
import numpy as np
import chart_studio
import chart_studio.plotly as py
import plotly.graph_objects as go

In [290]:
df = pd.read_excel("http://www.econ.yale.edu/~shiller/data/ie_data.xls", 
                   sheet_name='Data',
                   skiprows = range(1,7),
                   usecols = "A:V",
                   header=1)

In [291]:
# Cleaning:

df.dropna(subset=['Date'], inplace=True)
df.rename(columns={"Price":"real price", "Price.1":"Real TR Price", "TR CAPE": "P/E 10"}, inplace=True)
df['Date'] = pd.to_datetime(df['Date'].mul(100).astype(int), format='%Y%m').dt.to_period('m')

df = df[["Date", "real price", "Real TR Price", "CPI", "P/E 10"]]

df.head()

,Date,real price,Real TR Price,CPI,P/E 10
0,1871-01,99.544176,99.544176,12.464061,NaN
1,1871-02,97.900067,98.371438,12.844641,NaN
2,1871-03,98.828746,99.771312,13.034972,NaN
3,1871-04,105.464891,106.957430,12.559226,NaN
4,1871-05,110.649454,112.715639,12.273812,NaN


In [292]:
### BEREGNE NYE DATA
df["CPI yoy"] = df["CPI"].pct_change(12) * 100

df["Aksjer Vol."] = df["Real TR Price"].pct_change()
df["Aksjer Vol."] = (df["Aksjer Vol."].rolling(12).std() * np.sqrt(12))*100

df["Infl. Vol. *"] = df["CPI"].pct_change()
df["Infl. Vol. *"] = (df["Infl. Vol. *"].rolling(12).std() * np.sqrt(12))*100 * 5

df.dropna(inplace=True)

In [293]:
df

,Date,real price,Real TR Price,CPI,P/E 10,CPI yoy,Aksjer Vol.,Infl. Vol. *
120,1881-01,183.636477,3.397363e+02,9.419420,24.135057,-5.714633,10.677141,28.256563
121,1881-02,181.212339,3.364741e+02,9.514585,23.655503,-4.762059,11.237211,29.158643
122,1881-03,183.268233,3.415412e+02,9.514585,23.767713,-5.659953,11.219824,28.338685
123,1881-04,180.873272,3.383424e+02,9.609669,23.308850,-0.980597,11.678880,22.650466
124,1881-05,190.904409,3.584115e+02,9.514585,24.456686,1.010309,9.580874,17.409671
...,...,...,...,...,...,...,...,...
1809,2021-10,4506.727150,2.879192e+06,276.589000,40.638888,6.221869,6.534703,5.617948
1810,2021-11,4692.482762,3.001077e+06,277.948000,42.287407,6.809003,6.615392,4.705491
1811,2021-12,4685.512220,2.999845e+06,278.802000,42.197971,7.036403,6.367397,4.239004
1812,2022-01,4577.312664,2.930572e+06,281.148000,41.153302,7.479872,7.343874,4.364419


## Fordele PE10, Inflasjonsvol., og aksjevol i 12 like store kategorier

In [294]:
temp_bin_labels = list(np.arange(1,13))

df['cpi_bins'] = pd.qcut(df['CPI yoy'], 
                         q=12,
                        labels=temp_bin_labels)


df["bin_mean"] = np.nan
for i in temp_bin_labels:
    label = df[df["cpi_bins"] == i]["CPI yoy"].mean()   
    df.loc[df['cpi_bins'] == i, 'bin_mean'] = label
    
df

,Date,real price,Real TR Price,CPI,P/E 10,CPI yoy,Aksjer Vol.,Infl. Vol. *,cpi_bins,bin_mean
120,1881-01,183.636477,3.397363e+02,9.419420,24.135057,-5.714633,10.677141,28.256563,1,-8.684369
121,1881-02,181.212339,3.364741e+02,9.514585,23.655503,-4.762059,11.237211,29.158643,2,-2.643742
122,1881-03,183.268233,3.415412e+02,9.514585,23.767713,-5.659953,11.219824,28.338685,1,-8.684369
123,1881-04,180.873272,3.383424e+02,9.609669,23.308850,-0.980597,11.678880,22.650466,3,-0.158944
124,1881-05,190.904409,3.584115e+02,9.514585,24.456686,1.010309,9.580874,17.409671,4,1.006730
...,...,...,...,...,...,...,...,...,...,...
1809,2021-10,4506.727150,2.879192e+06,276.589000,40.638888,6.221869,6.534703,5.617948,10,5.483000
1810,2021-11,4692.482762,3.001077e+06,277.948000,42.287407,6.809003,6.615392,4.705491,11,7.999781
1811,2021-12,4685.512220,2.999845e+06,278.802000,42.197971,7.036403,6.367397,4.239004,11,7.999781
1812,2022-01,4577.312664,2.930572e+06,281.148000,41.153302,7.479872,7.343874,4.364419,11,7.999781


In [314]:
dataofinterest = ["Infl. Vol. *", "Aksjer Vol.", "P/E 10"]
plotdata = df.groupby(["bin_mean"])[dataofinterest].describe()
cols = ["#4582ec", "#f0ad4e", "#02b875"]
xs = plotdata.index/100

fig = go.Figure()

color_n = 0

for c in dataofinterest:
    fig.add_trace(go.Scatter(
            x=xs, 
            y=plotdata[c]["mean"],
            mode='lines',
            name = str(c),
            line=dict(width=2, color=cols[color_n]),
    ))
    
    color_n += 1
    
fig.update_layout(showlegend=False,
                  xaxis=dict(title="Inflasjon (år-over-år)",
                            showline=True,
                            showgrid=False,
                            showticklabels=True,
                            tickformat= '%',
                            #tickangle=-90,
                            linecolor="rgba(150,150,150,0.8)",
                            linewidth=1,
                            ticks="outside",
                            tickfont=dict(family="Arial", size=12, color="#6c757d"),
                            titlefont=dict(family="Arial", size=12, color="rgb(150,150,150)"),),
                  yaxis=dict(title="Gjennomsnittlig P/E10 og volatilitet",
                             titlefont=dict(family="Arial", size=12, color="rgb(150,150,150)"),
                            side="left",
                             linewidth=1,
                            linecolor="rgba(150,150,150,0.8)",
                            showgrid=False,
                            showticklabels=True,
                            tickfont=dict(family="Arial", size=12, color="rgba(150,150,150,0.6)")),
                  plot_bgcolor="rgba(0,0,0,0)", paper_bgcolor='rgba(0, 0, 0, 0)',
                  margin=dict(l=0, r=0, t=0, b=0),
                  hovermode = "x"
                 )

fig.update_xaxes(showspikes=True, spikecolor="rgba(150,150,150,0.8)", spikesnap="cursor", spikemode="across"),
fig.update_yaxes(showspikes=True, spikecolor="rgba(150,150,150,0.5)", spikethickness=2)


fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide',
                 spikedistance=1000, hoverdistance=100,
                 )


fig.show()